In [1]:
import chess.pgn
import os
import json
import math
import time

# ================= 配置区域 =================
# 输入文件路径
INPUT_PGN_PATH = "dataset/lichess_db_standard_rated_2025-11.pgn"

# 输出根目录
OUTPUT_ROOT = "dataset/2025-11"

# 进度记录文件 (用于断点续传)
STATE_FILE = "dataset/process_state.json"

# 缓冲区大小 (局数): 积攒多少局游戏写一次硬盘
# 建议 5000 - 20000 之间，取决于你的内存
BUFFER_SIZE = 10000 
# ===========================================

def get_time_control_category(tc_tag):
    if tc_tag == "-" or not tc_tag:
        return "Unknown"
    
    try:
        if "+" in tc_tag:
            base, increment = tc_tag.split("+")
            base = int(base)
            increment = int(increment)
        else:
            base = int(tc_tag)
            increment = 0
            
        # Lichess 官方公式: 预计时长 = 基础时间 + 40 * 加秒
        estimated_duration = base + (40 * increment)
        
        # 严格参照 Lichess 源代码 (Speed.scala) 的阈值
        if estimated_duration < 180:   # < 3分钟 (180s)
            return "Bullet"
        elif estimated_duration < 480: # < 8分钟 (480s) [注意：之前很多标准是600，Lichess是480]
            return "Blitz"
        elif estimated_duration < 1500: # < 25分钟 (1500s)
            return "Rapid"
        else:
            return "Classical"      # >= 25分钟
    except:
        return "Unknown"

def get_elo_category(white_elo, black_elo):
    """
    计算平均分并返回分段字符串 (每200分为一段)
    """
    try:
        w = int(white_elo)
        b = int(black_elo)
        avg = (w + b) / 2
        
        # 向下取整到最近的 200 分
        # 例如 1550 -> 1400, 1601 -> 1600
        floor_val = math.floor(avg / 200) * 200
        return f"{floor_val}-{floor_val + 199}"
    except:
        return "Unrated"

def save_state(offset, games_processed):
    """保存当前进度到 JSON"""
    with open(STATE_FILE, "w") as f:
        json.dump({"offset": offset, "games_processed": games_processed}, f)

def load_state():
    """读取上次的进度"""
    if os.path.exists(STATE_FILE):
        with open(STATE_FILE, "r") as f:
            return json.load(f)
    return {"offset": 0, "games_processed": 0}

def flush_buffer(buffer_dict):
    """
    将缓冲区的数据写入磁盘
    buffer_dict 结构: { "Blitz/1400-1599": [game_str1, game_str2...], ... }
    """
    for key, games in buffer_dict.items():
        if not games:
            continue
            
        tc_cat, elo_cat = key.split("/")
        
        # 构建目录路径: dataset/2025-11/Blitz/1400-1599/
        dir_path = os.path.join(OUTPUT_ROOT, tc_cat, elo_cat)
        os.makedirs(dir_path, exist_ok=True)
        
        # 追加写入 games.pgn
        file_path = os.path.join(dir_path, "games.pgn")
        
        # 使用 'a' (append) 模式
        with open(file_path, "a", encoding="utf-8") as f:
            for game_str in games:
                f.write(game_str + "\n\n")

# ================= 主程序逻辑 =================

def process_pgn():
    # 1. 加载断点
    state = load_state()
    start_offset = state["offset"]
    total_games = state["games_processed"]
    
    print(f"--- 任务启动 ---")
    if start_offset > 0:
        print(f"发现断点记录，将从字节位置 {start_offset} (约已处理 {total_games} 局) 继续...")
    else:
        print("从头开始处理...")

    # 内存缓冲区
    game_buffer = {} # Key: "Category/Elo", Value: List of game strings
    current_batch_count = 0
    
    start_time = time.time()

    # 2. 打开大文件
    with open(INPUT_PGN_PATH, encoding="utf-8", errors="ignore") as pgn_file:
        # 如果有断点，直接跳转
        pgn_file.seek(start_offset)
        
        while True:
            # 记录读取这局游戏之前的文件指针位置
            # 注意：read_game 读取完后指针会在下一局开头，
            # 但我们需要保存的是"下一批次开始的位置"，所以我们在 flush 时保存 tell()
            
            try:
                # 读取一局
                game = chess.pgn.read_game(pgn_file)
            except Exception as e:
                print(f"读取出错 (跳过): {e}")
                continue

            if game is None:
                break # 文件结束
            
            # 提取元数据
            headers = game.headers
            tc = headers.get("TimeControl", "?")
            w_elo = headers.get("WhiteElo", "?")
            b_elo = headers.get("BlackElo", "?")
            
            # 分类
            tc_cat = get_time_control_category(tc)
            elo_cat = get_elo_category(w_elo, b_elo)
            
            dict_key = f"{tc_cat}/{elo_cat}"
            
            # 将游戏对象转回 PGN 字符串 (这是最耗时的步骤之一，但在保存为PGN时无法避免)
            # 使用 exporter 稍微快一点，且可以不输出复杂的评论如果不需要的话
            # 这里我们使用标准的 str(game) 保留所有信息
            game_str = str(game)
            
            # 加入缓冲区
            if dict_key not in game_buffer:
                game_buffer[dict_key] = []
            game_buffer[dict_key].append(game_str)
            
            current_batch_count += 1
            total_games += 1
            
            # 3. 缓冲区满，写入磁盘并保存进度
            if current_batch_count >= BUFFER_SIZE:
                flush_buffer(game_buffer)
                
                # 获取当前文件指针位置 (这就是新的断点)
                current_offset = pgn_file.tell()
                save_state(current_offset, total_games)
                
                # 打印进度
                elapsed = time.time() - start_time
                speed = current_batch_count / elapsed if elapsed > 0 else 0
                print(f"已处理: {total_games} 局 | 当前位置: {current_offset} | 速度: {speed:.1f} 局/秒")
                
                # 清空缓冲区
                game_buffer = {}
                current_batch_count = 0
                start_time = time.time() # 重置计时器以计算瞬时速度

    # 4. 处理剩余的缓冲区
    if current_batch_count > 0:
        print("正在写入剩余数据...")
        flush_buffer(game_buffer)
        save_state(pgn_file.tell(), total_games)
        
    print("--- 处理完成！所有的对局已分类保存 ---")

# 运行处理
if __name__ == "__main__":
    # 确保根目录存在
    if not os.path.exists("dataset"):
        os.makedirs("dataset")
        
    process_pgn()

--- 任务启动 ---
从头开始处理...
已处理: 10000 局 | 当前位置: 23390096 | 速度: 248.4 局/秒
已处理: 20000 局 | 当前位置: 46647498 | 速度: 160.3 局/秒
已处理: 30000 局 | 当前位置: 69946537 | 速度: 129.1 局/秒
已处理: 40000 局 | 当前位置: 93365941 | 速度: 166.1 局/秒
已处理: 50000 局 | 当前位置: 116671126 | 速度: 252.9 局/秒
已处理: 60000 局 | 当前位置: 139876851 | 速度: 222.8 局/秒
已处理: 70000 局 | 当前位置: 163042783 | 速度: 216.7 局/秒
已处理: 80000 局 | 当前位置: 186139906 | 速度: 213.9 局/秒
已处理: 90000 局 | 当前位置: 209358433 | 速度: 141.2 局/秒
已处理: 100000 局 | 当前位置: 232454088 | 速度: 215.6 局/秒
已处理: 110000 局 | 当前位置: 255822270 | 速度: 244.3 局/秒
已处理: 120000 局 | 当前位置: 279047204 | 速度: 248.0 局/秒
已处理: 130000 局 | 当前位置: 302207905 | 速度: 247.1 局/秒
已处理: 140000 局 | 当前位置: 325392959 | 速度: 233.2 局/秒
已处理: 150000 局 | 当前位置: 348585538 | 速度: 223.8 局/秒
已处理: 160000 局 | 当前位置: 371716288 | 速度: 220.4 局/秒
已处理: 170000 局 | 当前位置: 394858349 | 速度: 214.0 局/秒
已处理: 180000 局 | 当前位置: 418167404 | 速度: 221.8 局/秒
已处理: 190000 局 | 当前位置: 441309214 | 速度: 196.9 局/秒
已处理: 200000 局 | 当前位置: 464562259 | 速度: 208.4 局/秒
已处理: 210000 局 | 当前位置: 48782381

KeyboardInterrupt: 